<a href="https://colab.research.google.com/github/tylerle29/Memotron/blob/main/Copy_of_NVIDIA_Nemotron_Nano_9B_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

trust_remote_code=True

In [ ]:
!pip install mamba-ssm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.6.post3-cp312-cp312-linux_x86_64.whl size=370558647 sha256=e6e1d10ffc02d0f251b45369779d532dc154de95148df2a2db4de39aa600356f
  Stored in directory: /root/.cache/pip/wheels/a9/66/b0/91003b49a34b45995d1b34c7fd2c70cd998a23560e66f6817a
Successfully built mamba-ssm


In [ ]:
'''
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/NVIDIA-Nemotron-Nano-9B-v2")

# Load model (on GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    "nvidia/NVIDIA-Nemotron-Nano-9B-v2",
    torch_dtype=torch.bfloat16,  # saves memory, works on GPU
    trust_remote_code=True,
    device_map="auto"
)

# Example text input
input_text = "Hello, how are you today?"

# Tokenize input
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output (limited to 50 tokens)
outputs = model.generate(**inputs, max_new_tokens=50)

# Decode output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Input:", input_text)
print("Output:", decoded_output)
'''

In [ ]:
!pip install flask flask-ngrok transformers torch -q
!pip install flask-ngrok

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Task
Replace `flask-ngrok` with `pyngrok` to expose the Flask application running in Google Colab via a public ngrok URL.

## Install pyngrok

### Subtask:
Add a cell to install the `pyngrok` library.


**Reasoning**:
The subtask is to install the `pyngrok` library. This can be done using pip in a new code cell.



In [ ]:
!pip install pyngrok

## Modify flask app

### Subtask:
Update the Flask application cell to remove `flask-ngrok` and integrate `pyngrok` to establish the tunnel.


**Reasoning**:
Update the Flask application code to use `pyngrok` instead of `flask-ngrok` to expose the application. This involves removing the old import and function call, importing `ngrok` from `pyngrok`, connecting to the default Flask port (5000) using `ngrok.connect()`, and printing the public URL.



**Reasoning**:
The previous attempt to connect with ngrok failed because it requires authentication. To resolve this, I need to add the ngrok authtoken before connecting. I will use the `ngrok.set_auth_token()` function for this purpose.



## Run the flask app with pyngrok

### Subtask:
Execute the modified Flask app cell to start the server and the ngrok tunnel using pyngrok.


**Reasoning**:
Execute the modified Flask app cell to start the server and the ngrok tunnel. The instructions indicate to run the cell containing the modified Flask application.



In [56]:
# Clear the CUDA cache
torch.cuda.empty_cache()

# Delete variables that might be holding onto GPU memory
# You might need to add other variable names here if you have more objects on the GPU
if 'model' in locals():
    del model
if 'pipe' in locals():
    del pipe
if 'pipe_smaller' in locals():
    del pipe_smaller
if 'input_ids' in locals():
    del input_ids
if 'outputs' in locals():
    del outputs
if 'tokenizer' in locals():
    del tokenizer

# Run Python's garbage collector
import gc
gc.collect()

print("GPU memory hopefully cleared.")

GPU memory hopefully cleared.


## Run the flask app with pyngrok

### Subtask:
Execute the modified Flask app cell to start the server and the ngrok tunnel using pyngrok.


**Reasoning**:
Execute the cell containing the modified Flask application code to start the server and the ngrok tunnel using pyngrok.



In [28]:
!pip install flask pyngrok

In [36]:
!pip install flask-cors

In [58]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

# Ensure you replace "YOUR_AUTHTOKEN" with your actual ngrok authtoken
ngrok.set_auth_token("")

app = Flask(__name__)

CORS(app)

device = "cuda" if torch.cuda.is_available() else "cpu"

print("Loading model, this may take a while...")
tokenizer = AutoTokenizer.from_pretrained(
    "nvidia/NVIDIA-Nemotron-Nano-4B‑v1.1",
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    "nvidia/NVIDIA-Nemotron-Nano-4B‑v1.1",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
print("Model loaded!")

@app.route("/api/respond", methods=["POST"])
def respond():
    data = request.json
    input_text = data.get("input", "")

    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=50)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return jsonify({
        "input": input_text,
        "output": decoded_output
    })


# Open a public tunnel on port 5000
# Ensure the app is running on port 5000
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)

# Run Flask (blocking)
app.run(port=5000)

Loading model, this may take a while...


HFValidationError: Repo id must use alphanumeric chars, '-', '_' or '.'. The name cannot start or end with '-' or '.' and the maximum length is 96: 'nvidia/NVIDIA-Nemotron-Nano-4B‑v1.1'.

## Summary:

## Data Analysis Key Findings

*   The `pyngrok` library was successfully installed.
*   Attempts to establish an ngrok tunnel using `pyngrok` failed due to an authentication error (`ERR_NGROK_4018`).
*   The authentication failure occurred because the `NGROK_AUTH_TOKEN` environment variable was not set in the Google Colab environment.
*   The Flask application server did not start successfully as the script terminated after the `ngrok.connect()` call failed.

## Insights or Next Steps

*   To successfully expose the Flask application via a public ngrok URL, the user must set the `NGROK_AUTH_TOKEN` environment variable in their Google Colab environment or secrets with a valid ngrok authentication token.
*   After setting the authtoken, re-executing the Flask application cell is required to attempt establishing the ngrok tunnel and starting the Flask server.
